# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770899876802                   -0.52    9.0         
  2   -2.772148331163       -2.90       -1.32    1.0    161ms
  3   -2.772170263302       -4.66       -2.46    1.0    140ms
  4   -2.772170650002       -6.41       -3.16    1.0    142ms
  5   -2.772170722672       -7.14       -4.11    2.0    198ms
  6   -2.772170722926       -9.59       -4.46    1.0    145ms
  7   -2.772170723012      -10.07       -5.49    1.0    148ms
  8   -2.772170723015      -11.49       -6.10    2.0    194ms
  9   -2.772170723015      -13.19       -6.89    1.0    153ms
 10   -2.772170723015      -13.82       -7.97    2.0    185ms
 11   -2.772170723015   +  -14.12       -8.08    2.0    179ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770678208154                   -0.53    9.0         
  2   -2.7

1.7735580830487492

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770572059580                   -0.53    9.0         
  2   -2.772049588303       -2.83       -1.29    1.0    138ms
  3   -2.772082629472       -4.48       -2.72    1.0    178ms
  4   -2.772083413406       -6.11       -3.76    2.0    160ms
  5   -2.772083417460       -8.39       -4.25    2.0    190ms
  6   -2.772083417806       -9.46       -5.21    1.0    145ms
  7   -2.772083417810      -11.34       -6.11    1.0    146ms
  8   -2.772083417811      -12.72       -6.71    2.0    192ms
  9   -2.772083417811   +  -14.51       -7.37    1.0    159ms
 10   -2.772083417811      -14.88       -7.78    1.0    165ms
 11   -2.772083417811   +  -14.27       -8.71    2.0    177ms

Polarizability via ForwardDiff:       1.7725349710326301
Polarizability via finite difference: 1.7735580830487492
